In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time

In [6]:
news_request = requests.get("https://news.kompas.com/")
news_content = news_request.content

# parse konten dengan beautifulsoup
news_bs4 = BeautifulSoup(news_content, 'html.parser')

# temukan semua elemen <li> dengan atribut yang ditentukan
news_bs4_titles = news_bs4.find_all(name="li", attrs={"class": "nav__item"})

categories = []
links = []

for title in news_bs4_titles:
    category = title.find('a').get_text()
    link = title.find('a')["href"]
    
    if not link.startswith('http'):
        link = "https://news.kompas.com/" + link
        
    categories.append(category)
    links.append(link)

df_bbc_links = pd.DataFrame({
    "Category": categories,
    "Link": links
})

df_bbc_links

,Category,Link
0,News,https://news.kompas.com?source=navbar
1,Pemilu,https://pemilu.kompas.com?source=navbar
2,IKN,https://ikn.kompas.com?source=navbar
3,Tekno,https://tekno.kompas.com?source=navbar
4,Otomotif,https://otomotif.kompas.com?source=navbar
5,Bola,https://bola.kompas.com?source=navbar
6,Lifestyle,https://lifestyle.kompas.com?source=navbar
7,Tren,https://www.kompas.com/tren?source=navbar
8,Lestari,https://lestari.kompas.com?source=navbar
9,Health,https://health.kompas.com?source=navbar


In [9]:
new_list = []

for index, row in tqdm(df_bbc_links.iterrows(), total=df_bbc_links.shape[0]):
    link = row['Link']
    response = requests.get(link)
    page_content = response.content
    
    page_soup = BeautifulSoup(page_content, 'html.parser')
    
    news_card = page_soup.find_all(attrs={"class": ["articleItem-box","articleHL-box"]})
    
    for card in news_card:
        catalog = {
            'headline': card.find(attrs={'class': "articleTitle"}).get_text() if card.find(attrs={"class": "articleTitle"}) else None,
            'description': card.find(attrs={'class': "articleLead"}).get_text() if card.find(attrs={'class': "articleLead"}) else None,
            'last_updated': card.find(attrs={"class": "articlePost-date"}).get_text() if card.find(attrs={"class": "articlePost-date"}) else None,
            'tag': card.find(attrs={"class": "articlePost-subtitle"}).get_text() if card.find(attrs={"class": "articlePost-subtitle"}) else None,
            "category": row['Category'],  # Ensure 'category' is a valid column name
            "link": link
        }
        new_list.append(catalog)
        
new_df = pd.DataFrame(new_list)

new_df.drop_duplicates(inplace=True)

new_df


100%|██████████| 17/17 [00:05<00:00,  3.02it/s]


,headline,description,last_updated,tag,category,link
0,Kronologi Sopir Truk Hantam Motor dan Mobil di...,None,1 November 2024,Megapolitan,News,https://news.kompas.com?source=navbar
1,"Momen Pertemuan Prabowo dan Jokowi di Solo, Di...",None,3 November 2024,Nasional,News,https://news.kompas.com?source=navbar
2,"Saat Lukman Pancing Gus Hans ""Serang"" Petahana...",None,3 November 2024,Surabaya,News,https://news.kompas.com?source=navbar
3,"Jembatan Ambruk di Kepulauan Aru, 44 Orang Ter...",None,3 November 2024,Regional,News,https://news.kompas.com?source=navbar
4,"PKL Bermunculan, Tepi Jalur Puncak Bogor Dipas...",None,3 November 2024,Bandung,News,https://news.kompas.com?source=navbar
...,...,...,...,...,...,...
379,"Persidangan Terdakwa Anak, Terlalu Sederhanakah?",None,24 Oktober 2024,Analisis,,https://news.kompas.com/#
380,Mekanisme Hukum Lelang Wilayah Izin Usaha Pert...,None,None,Konsultasi,,https://news.kompas.com/#
381,"Barang Gadai Hilang, Bagaimana Hukumnya?",None,None,Konsultasi,,https://news.kompas.com/#
382,Over Kredit Kendaraan Tanpa Persetujuan Perusa...,None,None,Konsultasi,,https://news.kompas.com/#


In [11]:
# Menyimpan DataFrame ke dalam file CSV
new_df.to_csv("kompas_news_data.csv", index=False, encoding="utf-8")

print("Data telah disimpan ke dalam file 'bbc_news_data.csv'")

Data telah disimpan ke dalam file 'bbc_news_data.csv'
